In [1]:
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import utils

data_directory = "/Users/nepal/Documents/synapthesis/synData6July"
lotti_fn = "export_lotti_veneto_2016_2018_giulio_v2.csv"
vincitori_fn = "export_vincitori_veneto_2016_2018_giulio_v2.csv"

lotti = pd.read_csv(path.join(data_directory, lotti_fn))
vincitori = pd.read_csv(path.join(data_directory, vincitori_fn))

lotti.data_inizio = pd.to_datetime(lotti.data_inizio, yearfirst=True)
lotti.data_fine = pd.to_datetime(lotti.data_fine, yearfirst=True)
lotti.data_inferita = pd.to_datetime(lotti.data_inferita, yearfirst=True)

In [2]:
utils.print_df_measures(lotti)

SHAPE: (1382247, 16)

DTYPES
id_lotto                         int64
id_pa                            int64
oggetto                         object
id_scelta_contraente           float64
importo                        float64
importo_base_asta              float64
importo_liquidato              float64
data_inizio             datetime64[ns]
data_fine               datetime64[ns]
data_inferita           datetime64[ns]
id_lsf                         float64
id_mod_realizz                 float64
id_forma_giuridica               int64
uber_forma_giuridica            object
cpv                            float64
cpv_vero                        object
dtype: object

COMPLETENESS percentages (100 means no NaNs)
id_lotto                100.000000
id_pa                   100.000000
oggetto                  99.987122
id_scelta_contraente     99.903273
importo                  99.603038
importo_base_asta         1.890834
importo_liquidato        69.557973
data_inizio              68.549036
data_fi

# Data preparation

## lotti dataframe

In [3]:
lotti = utils.replace_missing_value(lotti, "importo", "importo_base_asta")

importo values to substitute:           0.0012%


In [4]:
lotti = utils.replace_missing_value(lotti, "data_inizio", "data_inferita")

data_inizio values to substitute:           0.3145%


In [5]:
# drop non relevante table attributes
lotti = lotti.drop(columns=["oggetto", "importo_liquidato", "importo_base_asta", "data_inferita", "id_mod_realizz", "cpv_vero"])

per il momento scarto tutte le transazioni con almeno un missing value.
eventualmente quando non mi servirà la durata, tornerò indietro. 

In [6]:
lotti = lotti.dropna()

## cast to int64 cols now w/out np.nan

In [7]:
lotti.id_scelta_contraente = lotti.id_scelta_contraente.astype('int')
lotti.id_lsf = lotti.id_lsf.astype('int')
lotti.cpv = lotti.cpv.astype('int')
utils.print_df_measures(lotti)

SHAPE: (788453, 10)

DTYPES
id_lotto                         int64
id_pa                            int64
id_scelta_contraente             int64
importo                        float64
data_inizio             datetime64[ns]
data_fine               datetime64[ns]
id_lsf                           int64
id_forma_giuridica               int64
uber_forma_giuridica            object
cpv                              int64
dtype: object

COMPLETENESS percentages (100 means no NaNs)
id_lotto                100.0
id_pa                   100.0
id_scelta_contraente    100.0
importo                 100.0
data_inizio             100.0
data_fine               100.0
id_lsf                  100.0
id_forma_giuridica      100.0
uber_forma_giuridica    100.0
cpv                     100.0
dtype: float64


## vincitori dataframe


In [8]:
utils.print_df_measures(vincitori)

SHAPE: (917316, 4)

DTYPES
id_lotto                 int64
id_be                    int64
id_forma_giuridica       int64
uber_forma_giuridica    object
dtype: object

COMPLETENESS percentages (100 means no NaNs)
id_lotto                100.000000
id_be                   100.000000
id_forma_giuridica      100.000000
uber_forma_giuridica     99.893603
dtype: float64


In [9]:
vincitori = vincitori.dropna()

## lotti merge vincitori

In [10]:
df = lotti.merge(vincitori, on="id_lotto", how="inner", suffixes=("_acquirente", "_fornitore"))

In [11]:
utils.print_df_measures(df)

SHAPE: (755757, 13)

DTYPES
id_lotto                                    int64
id_pa                                       int64
id_scelta_contraente                        int64
importo                                   float64
data_inizio                        datetime64[ns]
data_fine                          datetime64[ns]
id_lsf                                      int64
id_forma_giuridica_acquirente               int64
uber_forma_giuridica_acquirente            object
cpv                                         int64
id_be                                       int64
id_forma_giuridica_fornitore                int64
uber_forma_giuridica_fornitore             object
dtype: object

COMPLETENESS percentages (100 means no NaNs)
id_lotto                           100.0
id_pa                              100.0
id_scelta_contraente               100.0
importo                            100.0
data_inizio                        100.0
data_fine                          100.0
id_lsf          

# feature extraction

In [12]:
# durata
df['durata'] = df.data_fine - df.data_inizio

In [13]:
# fatturato mediano annuo fornitori
supplier_rev = utils.compute_revenue_year(df, "id_be", "data_inizio").median(axis=1).rename("fatt_med_be")
df = df.merge(supplier_rev, on="id_be", how="left")

In [15]:
# erogato mediano annuo pa
buyer_rev = utils.compute_revenue_year(df, "id_pa", "data_inizio").median(axis=1).rename("erogato_med_pa")
df = df.merge(supplier_rev, on="id_be", how="left")

In [16]:
utils.print_df_measures(df)

SHAPE: (755757, 16)

DTYPES
id_lotto                                     int64
id_pa                                        int64
id_scelta_contraente                         int64
importo                                    float64
data_inizio                         datetime64[ns]
data_fine                           datetime64[ns]
id_lsf                                       int64
id_forma_giuridica_acquirente                int64
uber_forma_giuridica_acquirente             object
cpv                                          int64
id_be                                        int64
id_forma_giuridica_fornitore                 int64
uber_forma_giuridica_fornitore              object
durata                             timedelta64[ns]
fatt_med_be_x                              float64
fatt_med_be_y                              float64
dtype: object

COMPLETENESS percentages (100 means no NaNs)
id_lotto                           100.0
id_pa                              100.0
id_scelta_cont